# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be archieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-08 16:54:12] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=35685, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=678911865, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_

[2025-04-08 16:54:21 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-08 16:54:21 TP0] Init torch distributed begin.


[2025-04-08 16:54:22 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-08 16:54:22 TP0] Load weight begin. avail mem=78.58 GB


[2025-04-08 16:54:22 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-08 16:54:24 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.15s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.15s/it]

[2025-04-08 16:54:29 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-04-08 16:54:29 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-04-08 16:54:29 TP0] Memory pool end. avail mem=53.92 GB
[2025-04-08 16:54:29 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096


[2025-04-08 16:54:29] INFO:     Started server process [2638207]
[2025-04-08 16:54:29] INFO:     Waiting for application startup.
[2025-04-08 16:54:29] INFO:     Application startup complete.
[2025-04-08 16:54:29] INFO:     Uvicorn running on http://127.0.0.1:35685 (Press CTRL+C to quit)


[2025-04-08 16:54:29] INFO:     127.0.0.1:40996 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-08 16:54:30] INFO:     127.0.0.1:41006 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-08 16:54:30 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-08 16:54:34] INFO:     127.0.0.1:41020 - "POST /generate HTTP/1.1" 200 OK
[2025-04-08 16:54:34] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-08 16:54:34 TP0] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-08 16:54:35] INFO:     127.0.0.1:35922 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-04-08 16:54:35] Child process unexpectedly failed with an exit code 9. pid=2638791
[2025-04-08 16:54:35] Child process unexpectedly failed with an exit code 9. pid=2638724


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-08 16:54:43] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=34537, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=72198295, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_re

[2025-04-08 16:54:53 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-08 16:54:53 TP0] Init torch distributed begin.
[2025-04-08 16:54:53 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-08 16:54:53 TP0] Load weight begin. avail mem=59.50 GB


[2025-04-08 16:54:54 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-08 16:54:54 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.52s/it]


...

[2025-04-08 16:54:58 TP0] Init torch distributed begin.
[2025-04-08 16:54:58 TP0] Init torch distributed ends. mem usage=0.03 GB
[2025-04-08 16:54:58 TP0] Load weight begin. avail mem=35.86 GB


[2025-04-08 16:54:58 TP0] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.23s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.23s/it]

[2025-04-08 16:54:59 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=34.93 GB, mem usage=0.93 GB.
[2025-04-08 16:54:59 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-04-08 16:54:59 TP0] Memory pool end. avail mem=34.62 GB
[2025-04-08 16:54:59 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096


[2025-04-08 16:55:00] INFO:     Started server process [2640851]
[2025-04-08 16:55:00] INFO:     Waiting for application startup.
[2025-04-08 16:55:00] INFO:     Application startup complete.
[2025-04-08 16:55:00] INFO:     Uvicorn running on http://127.0.0.1:34537 (Press CTRL+C to quit)
[2025-04-08 16:55:00] INFO:     127.0.0.1:58732 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-08 16:55:01] INFO:     127.0.0.1:58736 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-08 16:55:01 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-08 16:55:06 TP0] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-04-08 16:55:08] INFO:     127.0.0.1:58742 - "POST /generate HTTP/1.1" 200 OK
[2025-04-08 16:55:08] The server is fired up and ready to roll!


[2025-04-08 16:55:08] INFO:     127.0.0.1:52844 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-04-08 16:55:08] Child process unexpectedly failed with an exit code 9. pid=2641439
[2025-04-08 16:55:08] Child process unexpectedly failed with an exit code 9. pid=2641338


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-08 16:55:18] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=39133, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=4575281, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_re

[2025-04-08 16:55:27 TP0] Casting torch.bfloat16 to torch.float16.


[2025-04-08 16:55:28 TP0] Casting torch.bfloat16 to torch.float16.
[2025-04-08 16:55:28 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-08 16:55:28 TP0] Init torch distributed begin.


[2025-04-08 16:55:28 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-08 16:55:28 TP0] Load weight begin. avail mem=42.35 GB


[2025-04-08 16:55:29 TP0] Ignore import error when loading sglang.srt.models.llama4. 


...

[2025-04-08 16:55:45 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192


[2025-04-08 16:55:46] INFO:     Started server process [2643474]
[2025-04-08 16:55:46] INFO:     Waiting for application startup.
[2025-04-08 16:55:46] INFO:     Application startup complete.
[2025-04-08 16:55:46] INFO:     Uvicorn running on http://127.0.0.1:39133 (Press CTRL+C to quit)


[2025-04-08 16:55:46] INFO:     127.0.0.1:53202 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-08 16:55:47] INFO:     127.0.0.1:53212 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-08 16:55:47 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-08 16:55:51] INFO:     127.0.0.1:53214 - "POST /generate HTTP/1.1" 200 OK
[2025-04-08 16:55:51] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-08 16:55:51 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-08 16:55:52] INFO:     127.0.0.1:46438 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-04-08 16:55:52] Child process unexpectedly failed with an exit code 9. pid=2643909


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-08 16:56:00] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=34280, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=893891732, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=

[2025-04-08 16:56:10 TP0] Casting torch.bfloat16 to torch.float16.


[2025-04-08 16:56:10 TP0] Casting torch.bfloat16 to torch.float16.
[2025-04-08 16:56:10 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-08 16:56:10 TP0] Init torch distributed begin.


[2025-04-08 16:56:11 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-08 16:56:11 TP0] Load weight begin. avail mem=44.41 GB


[2025-04-08 16:56:11 TP0] Ignore import error when loading sglang.srt.models.llama4. 


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.10it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.10it/s]

[2025-04-08 16:56:26 TP0] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=23.80 GB, mem usage=1.83 GB.


[2025-04-08 16:56:26 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-04-08 16:56:26 TP0] Memory pool end. avail mem=23.72 GB


[2025-04-08 16:56:26 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-08 16:56:27] INFO:     Started server process [2645424]
[2025-04-08 16:56:27] INFO:     Waiting for application startup.
[2025-04-08 16:56:27] INFO:     Application startup complete.
[2025-04-08 16:56:27] INFO:     Uvicorn running on http://127.0.0.1:34280 (Press CTRL+C to quit)
[2025-04-08 16:56:27] INFO:     127.0.0.1:50874 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-08 16:56:28] INFO:     127.0.0.1:50878 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-08 16:56:28 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-08 16:56:32 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-04-08 16:56:33] INFO:     127.0.0.1:50882 - "POST /generate HTTP/1.1" 200 OK
[2025-04-08 16:56:33] The server is fired up and ready to roll!


[2025-04-08 16:56:34] INFO:     127.0.0.1:40918 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-04-08 16:56:34] Child process unexpectedly failed with an exit code 9. pid=2646018
[2025-04-08 16:56:34] Child process unexpectedly failed with an exit code 9. pid=2645822


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionaly to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).